In [1]:
# 라이브러리 가져오기
import pandas as pd
import numpy as np

### 계층 색인

- 행, 열의 각 축에 대해 다중 단계(계층)를 지정하여 데이터에 차원을 설정
- 인덱스에 다차원 리스트를 전달하면 계층 색인을 지정할 수 있음
- 데이터 구조를 재배열하거나 pivot 테이블과 같은 그룹 기반 작업에 유용
- 재배열 메서드
    - stack() : 컬럼을 로우로 피벗
    - unstack() : 로우를 컬럼으로 피벗

In [4]:
# Series
# 인덱스에 다차원 리스트(아이템 2개)를 전달
# 다차원리스트[0] : 상위계층
# 상위계층 작성시 주의점 : 각 계층별로 속하는 하위계층 값의 개수만큼 계층명 작성
# 상위계층 리스트 개수 = 하위계층 리스트 개수
# 다차원리스트[1] : 하위계층
# 실습)
# 로우 인덱스 - 상위계층 : a, b, c, d
# 하위계층 : a(1, 2, 3), b(1, 2), c(1, 2, 3, 4), d(1)
s1 = pd.Series(np.arange(10),
              index=[['a','a','a','b','b','c','c','c','c','d'],
                    [1,2,3,1,2,1,2,3,4,1]])
s1

a  1    0
   2    1
   3    2
b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
d  1    9
dtype: int32

In [5]:
# 상위 계층에 접근
# 계층 색인이 적용된 객체에 상위 인덱스에 접근 : 일반적인 Series 인덱싱으로 부분집합
s1['c']

1    5
2    6
3    7
4    8
dtype: int32

In [6]:
# 계층색인에 대한 슬라이싱 : 마지막 라벨인덱스도 포함('b':'d'로 입력시 b, c, d 조회)
s1['b':'d']

b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
d  1    9
dtype: int32

In [8]:
# 하위 계층에 접근
# a, b, c, d에서 하위 계층 인덱스가 2인 요소 조회
s1[:,2]

a    1
b    4
c    6
dtype: int32

In [10]:
# 하위 계층에 슬라이싱 : loc 메서드를 이용하여 슬라이싱
# 상위 계층이 c이고 하위 계층을 2~4까지 슬라이싱(마지막 포함)
s1['c'].loc[2:4]

2    6
3    7
4    8
dtype: int32

In [11]:
# unstack() 메서드 : 최하위(기본동작)에 있는 로우 계층을 컬럼으로 적용하여 위로 올림
# Series 객체를 DataFrame 객체로 재배열 할 수 있음
# NaN: 기존에 없던 로우 계층에 대한 값
s1.unstack

<bound method Series.unstack of a  1    0
   2    1
   3    2
b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
d  1    9
dtype: int32>

In [12]:
# 재배열 적용시 레벨 지정 가능
# 상위 계층을 컬럼으로 적용하여 위로 올리기
s1.unstack(0)

,a,b,c,d
1,0.0,3.0,5.0,9.0
2,1.0,4.0,6.0,NaN
3,2.0,NaN,7.0,NaN
4,NaN,NaN,8.0,NaN


In [15]:
# stack() 메서드 : 컬럼에 있던 값을 인덱스의 하위로 내려서 재배열
# DataFrame을 Series로 만들때 사용
s1.unstack().stack();
s1

a  1    0
   2    1
   3    2
b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
d  1    9
dtype: int32

In [52]:
# DataFrame 생성
# 구조 4*5
# 로우인덱스 : 상위(2017, 2018) / 하위(모든 상위 인덱스에 대해 동일하게 'a', 'b')
# 컬럼인덱스 : 상위(서울, 경기) / 하위(서울-강남, 잠실/ 경기-분당, 수원, 판교)
# 값 : 1씩 증가하는 20개
df = pd.DataFrame(np.arange(20).reshape(4,5),
                 index = [[2017,2017,2018,2018],
                         ['a','b','a','b']],
                       columns = [['서울','서울',
                                   '경기','경기','경기'],
                                  ['강남','잠실','분당','수원','판교']])

In [39]:
df

서울      경기        
        강남  잠실  분당  수원  판교
2017 a   0   1   2   3   4
     b   5   6   7   8   9
2018 a  10  11  12  13  14
     b  15  16  17  18  19

In [40]:
df['경기'].loc[:,'분당':'수원']

분당  수원
2017 a   2   3
     b   7   8
2018 a  12  13
     b  17  18

In [41]:
# 컬럼 최하위를 로우의 하위계층으로 재배열 : stack()
df.unstack().stack()

경기          서울    
        분당  수원  판교  강남  잠실
2017 a   2   3   4   0   1
     b   7   8   9   5   6
2018 a  12  13  14  10  11
     b  17  18  19  15  16

In [50]:
# 계층으 ㅣ인덱스번호 또는 라벨을 사용하여 상 하위간 교환
# swaplevel(key1, key, axis=0(기본값))
# axis가 0인 경우는 row간의 상하위 개념을 바꿔준다.
df.swaplevel(1,0)

서울      경기        
        강남  잠실  분당  수원  판교
a 2017   0   1   2   3   4
b 2017   5   6   7   8   9
a 2018  10  11  12  13  14
b 2018  15  16  17  18  19

In [51]:
# axis=1은 로우 계층변동이 아닌 컬럼 계층변동
df.swaplevel(1,0,axis=1)

강남  잠실  분당  수원  판교
        서울  서울  경기  경기  경기
2017 a   0   1   2   3   4
     b   5   6   7   8   9
2018 a  10  11  12  13  14
     b  15  16  17  18  19

### 객체 복사

- 할당기호(=) / 슬라이싱 : 원본과 상호 종속적인 복사본 객체 생성(얕은복사)
- obj.copy() : 원본과 독립적인 복사본 객체 생성(깊은복사)

In [58]:
# 모든 값이 1인 5*1 구조의 Series생성
s1 = pd.Series(np.ones(5))
s1

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
dtype: float64

In [59]:
# 할당기호(=)로 복사 : 얕은복사
s2 = s1

In [60]:
s2[0] = 10
s2

0    10.0
1     1.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [62]:
# 원본객체 변동여부 확인
s1

0    10.0
1     1.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [63]:
# 원본 객체의 아이템 수정
s1[1] = 3

In [64]:
# 복사본 s2객체 확인
s2

0    10.0
1     3.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [66]:
# 슬라이싱으로 복사(일반 파이썬 -> 깊은복사, Pandas-> 얕은복사로 취급)
s3 = s1[:]
s3

0    10.0
1     3.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [69]:
s3[2] = -9
s3

0    10.0
1     3.0
2    -9.0
3     1.0
4     1.0
dtype: float64

In [70]:
# 원본과 s2 확인
print(s1)
print(s2)

0    10.0
1     3.0
2    -9.0
3     1.0
4     1.0
dtype: float64
0    10.0
1     3.0
2    -9.0
3     1.0
4     1.0
dtype: float64


In [71]:
# 원본변경
s1[3]=33
s1

0    10.0
1     3.0
2    -9.0
3    33.0
4     1.0
dtype: float64

In [73]:
# 복사본 객체 확인
print(s2)
print(s3)

0    10.0
1     3.0
2    -9.0
3    33.0
4     1.0
dtype: float64
0    10.0
1     3.0
2    -9.0
3    33.0
4     1.0
dtype: float64


In [74]:
# copy()를 이용한 복사(깊은복사)
s4 = s1.copy()
# s4 수정(s1 영향x)
s4[4] = 44
s4

0    10.0
1     3.0
2    -9.0
3    33.0
4    44.0
dtype: float64

In [75]:
# 원본 확인
s1

0    10.0
1     3.0
2    -9.0
3    33.0
4     1.0
dtype: float64

### 정렬

- obj.sort_index() : 인덱스를 기준으로 정렬 (기본값은 ascending=True, 오름차순 정렬)
    - DataFrame 
        - axis = 0 : 기본값, 로우 인덱스 기준으로 정렬
        - axis = 1 : 컬럼 인덱스 기준으로 정렬
- obj.sort_values() : 값을 기준으로 정렬
    - DataFrame 
        - by : 정렬의 기준이 되는 인덱스 값 전달
        - axis = 0 : 기본값, 컬럼을 기준으로 로우 인덱스를 정렬하며 기준값으로 by에 인덱스 컬럼 레벨 또는 컬럼명 전달
        - axis = 1 : 로우 인덱스를 기준으로 컬럼 라벨을 정렬하며 기준값으로 by에 레벨 또는 라벨명 전달

In [76]:
# Series 생성
# 값과 인덱스라벨이 순서대로 들어가지 않은 Series
s1 = pd.Series([2,3,1,7,0], index=list('gacfd'))
s1

g    2
a    3
c    1
f    7
d    0
dtype: int64

In [88]:
s1.sort_index()

a    3
c    1
d    0
f    7
g    2
dtype: int64

In [89]:
# DataFrame 생성
# 4 x 5, 무작위 정수
# 로우/컬럼 인덱스도 순서가 없는 값 지정
df1 = pd.DataFrame(np.random.randint(20, size=(4,5)),
                  index=list('hcae'), columns=list('EAFCD'))

In [90]:
df1

,E,A,F,C,D
h,6,13,2,8,4
c,16,8,10,4,2
a,4,10,17,14,12
e,10,2,19,12,14


In [94]:
# row 인덱스 기준으로 오름차순 정렬(axis=0)
df1.sort_index()

,E,A,F,C,D
a,4,10,17,14,12
c,16,8,10,4,2
e,10,2,19,12,14
h,6,13,2,8,4


In [96]:
# row 인덱스 기준으로 내림차순 정렬
df1.sort_index(ascending=False)

,E,A,F,C,D
h,6,13,2,8,4
e,10,2,19,12,14
c,16,8,10,4,2
a,4,10,17,14,12


In [99]:
df1.sort_index(axis=1).sort_index()

,A,C,D,E,F
a,10,14,12,4,17
c,8,4,2,16,10
e,2,12,14,10,19
h,13,8,4,6,2


In [103]:
# 컬럼 기준으로 내림차순 정렬 후, 로우 기준으로 오름차순 정렬
df1.sort_index(axis=0).sort_index(axis=1, ascending=False)

,F,E,D,C,A
a,17,4,12,14,10
c,10,16,2,4,8
e,19,10,14,12,2
h,2,6,4,8,13


In [112]:
# 값 기준으로 정렬
# 컬럼 D의 값을 오름차순으로 정렬
# sor_values(axis=0) : 기본동작(정렬 결과 행에 반영/정렬 기준은 D컬럼)
df1.sort_values(axis=0, by='D')

,E,A,F,C,D
c,16,8,10,4,2
h,6,13,2,8,4
a,4,10,17,14,12
e,10,2,19,12,14


In [114]:
df1.sort_values(by='A', ascending=False)

,E,A,F,C,D
h,6,13,2,8,4
a,4,10,17,14,12
c,16,8,10,4,2
e,10,2,19,12,14


In [ ]:
# 인덱스라벨 c의 값을 오름차순으로 정렬
# 결과적으로 정렬되는 대상 : 컬럼
# 정렬의 기준 : 로우 레이블

In [ ]:
# 두 개의 컬럼에 대해 정렬 : 리스트로 묶어서 by의 인자값으로 전달
# 정렬 우선 순위 : 차례대로 1순위 > 2순위(1순위 동점시 2순위로 넘어감)
# 1순위 정렬 후 1순위의 컬럼의 동점값에 대해 2순위 컬럼의 값이 재정렬
# 동일한 값이 아니라면 무조건 1순위 컬럼이 우선

In [122]:
df1.sort_values(by=['e','a'],axis=1,ascending=False)

,F,D,C,E,A
h,2,4,8,6,13
c,10,2,4,16,8
a,17,12,14,4,10
e,19,14,12,10,2


In [118]:
# 한 행은 내림차순, 한 행은 오름차순으로 할 경우
# ascending=[True, False]와 같이 값을 따로 지정
df1.sort_values(by=['e','a'], axis=1,ascending=[False,True])

,F,D,C,E,A
h,2,4,8,6,13
c,10,2,4,16,8
a,17,12,14,4,10
e,19,14,12,10,2


### 연습문제

#### 1. 아래의 데이터프레임을 생성하세요

- 학생들의 점수는 50 이상 100 미만의 무작위 정수 값을 생성하여 사용

<img src="img/df_sort_practice1.png" width=250 align='left'>

In [197]:
# DataFrame 생성
arr = np.random.randint(50,100,(5,4))
df = pd.DataFrame(np.array(arr), index=['kim','park','Lee','Jung','Moon'],
                 columns=[[2016,2016,2017,2017],['영어','수학','영어','수학']])
# row index 이름설정
df.index.set_names('학생명', inplace=True)

In [198]:
df

2016     2017    
       영어  수학   영어  수학
학생명                   
kim    92  76   74  69
park   95  51   82  63
Lee    72  71   72  95
Jung   64  55   82  52
Moon   64  63   90  84

In [199]:
# 컬럼 인덱스 이름 설정
df.columns.set_names(['연도','과목명'], inplace=True)

In [200]:
df

연도   2016     2017    
과목명    영어  수학   영어  수학
학생명                   
kim    92  76   74  69
park   95  51   82  63
Lee    72  71   72  95
Jung   64  55   82  52
Moon   64  63   90  84

#### 2. 2016년 데이터만 별도의 데이터프레임으로 분리저장(깊은복사)

In [201]:
df2016 = df[2016].copy()
df2016

과목명,영어,수학
학생명,,
kim,92,76
park,95,51
Lee,72,71
Jung,64,55
Moon,64,63


#### 4. 2016년 데이터의 과목별 점수에 대해 영어가 높은순서로 정렬, 영어 점수 동점자의 경우 수학점수가 낮은순서로 정렬

In [202]:
df2016.sort_values(by=['영어','수학'], ascending=[False,True])

과목명,영어,수학
학생명,,
park,95,51
kim,92,76
Lee,72,71
Jung,64,55
Moon,64,63


### 데이터 합치기

- merge
    - pandas 객체의 메서드로 pandas.merge(df1, df2 ...)로 사용
    - 두 개의 데이터프레임에 대해 특정 컬럼을 기준으로 합치기
    - 주요 파라미터
        - how : 합치는 방식으로 inner(기본값), left, right, outer 방식 존재
        - on : 합치는 기준으로 두 개의 데이터프레임에 공통으로 존재하는 컬럼명을 사용해야함 (기본값=None)

In [210]:
# 고객번호와 고객명을 저장하고
df1 = pd.DataFrame({'no':[30,31,32,33,34],'name':['파이썬','장고','팬더스','넘파이','주피터']})
df1

,no,name
0,30,파이썬
1,31,장고
2,32,팬더스
3,33,넘파이
4,34,주피터


In [205]:
# 고객 번호와 주문수량을 저장하고 있는 데이터프레임 생성
# 다른 고객번호(no)를 일부 지정
df2 = pd.DataFrame({'no':[30,33,40,41],'amount':[100,130,40,60]})
df2

,no,amount
0,30,100
1,33,130
2,40,40
3,41,60


In [217]:
# 가장 기본적인 merge 방식 : 공통 컬럼을 기준으로 두 df에 모두 존재하는 교집합만을 추출
# 기본 동작 : how='inner'
# df1 & df2의 공통컬럼 : no
# 공통 컬럼 내부의 공통 값 : 30, 33
# df1 : 30 ~ 34
# df2 : 30, 33, 40, 41
pd.merge(df1, df2)

,no,name,amount
0,30,파이썬,100
1,33,넘파이,130


In [218]:
#outer : 결합 기준으로 공통 컬럼을 사용함
# 그러나 교집합이 아닌 부분도 출력, 대신 누락된 부분은 NaN으로 처리
# 합집합 형태로 만듬
pd.merge(df1, df2, how='outer')

,no,name,amount
0,30,파이썬,100.0
1,31,장고,NaN
2,32,팬더스,NaN
3,33,넘파이,130.0
4,34,주피터,NaN
5,40,NaN,40.0
6,41,NaN,60.0


In [219]:
# left : 첫 번째로 전달한 df1만 모두 살리고, df2에서는 df1과 공통된 부분만 남김
pd.merge(df1, df2, how='left')

,no,name,amount
0,30,파이썬,100.0
1,31,장고,NaN
2,32,팬더스,NaN
3,33,넘파이,130.0
4,34,주피터,NaN


In [220]:
# right : 두 번째 데이터프레임의 보존
# df1에서는 df2와 겹치는 데이터만 가져옴
pd.merge(df1, df2, how='right')

,no,name,amount
0,30,파이썬,100
1,33,넘파이,130
2,40,NaN,40
3,41,NaN,60


In [221]:
# 공통 컬럼이 두 개 이상인 경우
# 고객명, 날짜, 정보를 저장하고 있는 데이터프레임 생성
df3 = pd.DataFrame({'고객명':['파이썬', '장고', '팬더스', ],'날짜':['2019-10-09','2019-10-11','2019-12-08'],
                   '정보':['010','011','019']})
df3

,고객명,날짜,정보
0,파이썬,2019-10-09,010
1,장고,2019-10-11,011
2,팬더스,2019-12-08,019


In [223]:
# 고객명, 정보를 저장하고 있는 데이터프레임 생성
df4 = pd.DataFrame({'고객명':['파이썬', '팬더스', '넘파이'],
                   '정보':['F','M','M']})
df4

,고객명,정보
0,파이썬,F
1,팬더스,M
2,넘파이,M


In [227]:
# 공통된 이름을 가진 컬럼이 두개인 경우 파라미터 없이 merge를 실행하면...
pd.merge(df3, df4)

,고객명,날짜,정보


In [228]:
# on 파라미터 : 공통된 이름을 가진 컬럼이 여럿인 경우 결합 기준을 지정
# 합칠 때 key로 사용할 컬럼 : '고객명'
# 공통컬럼 결과 : 고객명, 정보 -> 정보 컬럼 결과를 확인
# 기본동작으로 merge : inner
pd.merge(df3, df4, on='고객명')

,고객명,날짜,정보_x,정보_y
0,파이썬,2019-10-09,010,F
1,팬더스,2019-12-08,019,M


In [232]:
# left_on, right_on : 두 개의 데이터프레임에 대하여 서로 다른 기준 컬럼명을 지정
# 예) 동일한 속성의 자료를 저장하는 컬럼인데 표기하는 이름이 다른 경우
# 고객이름, 날짜, 구매금액을 저장하고 있는 데이터프레임 생성
df1 = pd.DataFrame({'고객이름':['파이썬','팬더스','주피터'],
                  '날짜':['2019-12-08','2019-10-09','2019-11-21'],
                  '구매금액':[1,2,3]})
df1

,고객이름,날짜,구매금액
0,파이썬,2019-12-08,1
1,팬더스,2019-10-09,2
2,주피터,2019-11-21,3


In [234]:
# 고객명, 성별을 저장하고 있는 데이터프레임을 생성
df2 = pd.DataFrame({'고객명':['파이썬','팬더스'],'성별':['F','M']})
df2

,고객명,성별
0,파이썬,F
1,팬더스,M


In [237]:
# left에서 지정, right에서 지정
tmp = pd.merge(df1,df2, left_on='고객이름', right_on='고객명')
tmp

,고객이름,날짜,구매금액,고객명,성별
0,파이썬,2019-12-08,1,파이썬,F
1,팬더스,2019-10-09,2,팬더스,M


In [238]:
# drop 메서드를 이용해 컬럼 삭제, axis=1인 경우 컬럼삭제, 0이면 로우 삭제
tmp.drop('고객명', axis=1, inplace=True)
tmp

,고객이름,날짜,구매금액,성별
0,파이썬,2019-12-08,1,F
1,팬더스,2019-10-09,2,M


#### concat

- 특정 key를 기준으로 데이터를 합치는 것이 아니라 행, 열 기준으로 데이터를 연결
- 주요 파라미터
    - axis : 0 / 행 방향(기본값)이며 컬럼을 key로 합치고, 1 / 열 방향으로 로우를 key로 합침
    - join : 데이터프레임끼리 연결할 때 합치는 방법으로 outer(기본값), inner 방식 존재
    - ignore_index : 합친 후 기존 인덱스를 유지 또는 새로운 인덱스를 지정

In [240]:
# 공통 인덱스 라벨을 가지는 Series 2개 생성
s1 = pd.Series([1,2,3], index=list('abc'))
s2 = pd.Series([5,6,7,8],index=list('abfh'))
print(s1);print(s2)

a    1
b    2
c    3
dtype: int64
a    5
b    6
f    7
h    8
dtype: int64


In [242]:
# 두 Series간 연결
# 기본 : axis=0(행 방향으로 연결)
# 첫 번째로 전달된 객체가 위에, 두 번쨰로 전달된 객체가 아래로 추가(연결)
# 인덱스 라벨은 추가 연결
pd.concat([s1,s2])

a    1
b    2
c    3
a    5
b    6
f    7
h    8
dtype: int64

In [243]:
# 새로운 인덱스로 초기화하기
pd.concat([s1,s2], ignore_index=True)

0    1
1    2
2    3
3    5
4    6
5    7
6    8
dtype: int64

In [245]:
# 열 방향 연결 -> 두 개의 Series를 연결하면 DataFrame으로 만들기
# 길이가 다른 경우
pd.concat([s1,s2], axis=1)

D:\jjw\anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,0,1
a,1.0,5.0
b,2.0,6.0
c,3.0,NaN
f,NaN,7.0
h,NaN,8.0


In [249]:
# 열 이름을 설정하면서 붙여주기 : keys 파라미터에 컬럼명을 리스트로 전달
pd.concat([s1,s2], axis=1, keys=['c1', 'c2'], sort=False)

,c1,c2
a,1.0,5.0
b,2.0,6.0
c,3.0,NaN
f,NaN,7.0
h,NaN,8.0


In [250]:
# 두 개의 데이터프레임 연결
# 고객명, 날짜 구매금액
df1 = pd.DataFrame({'고객명':['김파이썬', '이장고', '박팬더스'],
                   '날짜':['2019-10-09', '2019-10-10', '2019-10-11'],
                   '구매금액':[1, 2, 3]})

# 고객명, 성별
df2 = pd.DataFrame({'고객명':['김파이썬', '최넘파이'],
                   '성별':['F', 'M']})

In [251]:
df1

,고객명,날짜,구매금액
0,김파이썬,2019-10-09,1
1,이장고,2019-10-10,2
2,박팬더스,2019-10-11,3


In [252]:
df2

,고객명,성별
0,김파이썬,F
1,최넘파이,M


In [254]:
# 행 방향으로 데이터 프레임 연결(행 갯수 증가)
pd.concat([df1, df2], sort=False)

,고객명,날짜,구매금액,성별
0,김파이썬,2019-10-09,1.0,NaN
1,이장고,2019-10-10,2.0,NaN
2,박팬더스,2019-10-11,3.0,NaN
0,김파이썬,NaN,NaN,F
1,최넘파이,NaN,NaN,M


In [255]:
pd.concat([df1,df2], axis=1, sort=False)

,고객명,날짜,구매금액,고객명,성별
0,김파이썬,2019-10-09,1,김파이썬,F
1,이장고,2019-10-10,2,최넘파이,M
2,박팬더스,2019-10-11,3,NaN,NaN


In [256]:
df3 = pd.DataFrame({'지역':['서울', '부산', '제주'],
                   '직업':['웹개발자', '회계사', '회사원']})
df3

,지역,직업
0,서울,웹개발자
1,부산,회계사
2,제주,회사원


In [257]:
# 전제조건, df1의 데이터와 df3의 데이터가 순서상 알맞은것이 확정적인 경우

In [258]:
pd.concat([df1,df3], axis=1)

,고객명,날짜,구매금액,지역,직업
0,김파이썬,2019-10-09,1,서울,웹개발자
1,이장고,2019-10-10,2,부산,회계사
2,박팬더스,2019-10-11,3,제주,회사원
